In this program we will solve a reaction diffusion equation based on your input. 
The equation we work with is as follows:

# Diffusion equation: 
\begin{equation}
    u_{i,k+1} - D(\Delta t)\frac{u_{i-1,k+1}-2u_{i,k+1}+u_{i+1,k+1}}{2(\Delta x)^2}
    = u_{i,k} + D(\Delta t)\frac{u_{i-1,k}-2u_{i,k}+u_{i+1,k}}{2(\Delta x)^2} + (\Delta t)f(x_i,t_k,u_{i,k})
\end{equation}

In [1]:
using Plots

In [2]:
abstract type BoundaryCondition
end

type Dirichlet <: BoundaryCondition 
    ū₀::Float64
    ūₗ::Float64
end

type Neumann <: BoundaryCondition
    ∂ū₀::Float64
    ∂ūₗ::Float64
end 

type Periodic <: BoundaryCondition 
end 

In [3]:
type Discretization
    Nₓ::Int64
    Δx::Float64
    Nₜ::Int64
    Δt::Float64
end 

type SpaceTime
    L::Float64 #spacial extent 
    tf::Float64 #time span of simulation 
end

In [4]:
"""
Determine the number of spatial unknowns at each time step.
"""
function nb_spatial_unknowns(discretization::Discretization, bc::BoundaryCondition)
    if typeof(bc) == Neumann
        return discretization.Nₓ
    elseif typeof(bc) == Dirichlet
        return discretization.Nₓ - 2
    elseif typeof(bc) == Periodic
        return discretization.Nₓ - 1
    end
end

nb_spatial_unknowns

In [70]:
"""
Create tridiagonal matrix.
"""
function build_tri_diagonal_matrix(discretization::Discretization, λ::Float64, bc::BoundaryCondition)
    # determine number of unknowns based on boundary condition
    matrix_dim = nb_spatial_unknowns(discretization, bc)
    
    A = zeros(matrix_dim, matrix_dim)
    
    for i = 1:matrix_dim
        A[i, i] = 1 + 2 * λ
    end
        
    for i = 1:matrix_dim - 1
        A[i  , i+1] = -λ
        A[i+1, i  ] = -λ
    end
    
    if typeof(bc) == Periodic
        A[1, end] = -λ
        A[end, 1] = -λ
    end
        
    if typeof(bc) == Neumann
        A[1, 2] = -2 * λ
        A[end, end - 1] = -2 * λ
    end 
    
    return A
        
end

build_tri_diagonal_matrix

In [69]:
"""
Arguments:
* f: reaction term f(x, t, u)
* u₀: initial condition u₀(x)
* bc::BoundaryCondition: boundary condition
* D::Float64: diffusion coefficient
* Nₓ::Int: number of spatial discretization points
* st::SpaceTime: space-time over which solution to PDE is approximated
* sample_time::Float64: stores u every sample_time time steps
"""
function solve_rxn_diffn_eqn(f, u₀, bc::BoundaryCondition, D::Float64, Nₓ::Int, st::SpaceTime, sample_time::Float64)
    # compute Δx, Nₜ, Δt below.
    discretization = Discretization(Nₓ, NaN, 0, NaN)
    
    # ensure that the boundary condition is consistent with the initial condition
    if isa(bc, Dirichlet)
        @assert(bc.ū₀ ≈ u₀(0.0), 
            "The initial condition is inconsistent with Dirichlet boundary condition.")
        @assert(bc.ūₗ ≈ u₀(st.L), 
            "The initial condition is inconsistent with Dirichlet boundary condition.")
    elseif isa(bc, Periodic)
        @assert(u₀(0) ≈ u₀(st.L), 
            "The initial condition is inconsistent with periodic boundary conditions.")
    elseif isa(bc, Neumann)
        #TODO
    end
  
    # discretize space.
    x = collect(linspace(0, st.L, discretization.Nₓ)) # includes end points!
    discretization.Δx = x[2] - x[1]
    @printf("%d points in x-discretization. dx = %f\n", discretization.Nₓ, discretization.Δx)

    # discreteize time.
    discretization.Δt = discretization.Δx ^ 2
    discretization.Nₜ = ceil(Int, st.tf / discretization.Δt)
    @printf("%d points in t-discretization. dt = %f\n", discretization.Nₜ, discretization.Δt)
    
    # nondimensional parameter involved in discreteization
    λ = (D * discretization.Δt) / (2 * (discretization.Δx) ^ 2)
    
    # build tri-diagonal matrix
    A = build_tri_diagonal_matrix(discretization, λ, bc)
    
    nb_unknowns = nb_spatial_unknowns(discretization, bc)
    
    # initialize u at k = 0
    # this u is the value of u(x, t) when t = k - 1 over discretized x points.
    u = u₀.(x)
    
    # trim u and x so that i index corresponds to xᵢ and uᵢ,ₖ
    if typeof(bc) == Neumann
        # all are unknown, no trimming required
    elseif typeof(bc) == Dirichlet
        u = u[2:end-1]
        x = x[2:end-1]
    elseif typeof(bc) == Periodic
        u = u[1:end-1]
        x = x[1:end-1]
    end
    @assert(length(u) == nb_unknowns)
    @assert(length(x) == nb_unknowns)
 
    # initialize right-hand side of matrix eqn. solved at each time step.
    rhs = zeros(Float64, nb_unknowns)
    
    # initialize u_sample and t_sample to be filled below with specific time steps for graphing purposes 
    num_samples = ceil(Int, st.tf / sample_time) # determines size of u_sample and t_sample
    sample_freq = floor(Int, discretization.Nₜ / (num_samples - 1)) # don't count the 0 sample
    u_sample = zeros(Float64, nb_unknowns, num_samples) # nb_unknowns = size of u, num_samples, how many u's we will store 
    t_sample = zeros(Float64, num_samples)
    sample_counter = 1 # for if loop below to identify column of u_sample, increments below after u_sample is imput 
 
    # take time steps.
    for k = 1:discretization.Nₜ
       
        # time here, inside the loop
        t = discretization.Δt * k
        
        # build right-hand side vector of matrix eqn.
        for i = 1:nb_unknowns
            # contribution from first order discretization of time derivative
            rhs[i] = u[i]
            # contribution from reaction term
            rhs[i] += f(x[i], t, u[i]) * discretization.Δt
            # TODO contribution from advection 
            # contribution from spatial Laplacian
            if (i != 1) && (i != nb_unknowns)
                rhs[i] += λ * (u[i - 1] - 2 * u[i] + u[i + 1])
            end
         end
        
         # based on boundary condition, handle first and last component of Laplacian.
         if typeof(bc) == Neumann
         # -4 and 4 come from making "ghost points" as described by Gustafson (u₀ and uₙ₊₁ are considered our ghost points)
         # ∂ū₀ = (u₂ - u₀)/(2 * Δx) which means u₀ = -2 * Δx * ∂ū₀ + u₂
         # substitute u₀ in the right hand side of the diffusion equation at i = 1 to get the equation below for rhs[1] 
         # by the same logic, use uₙ₊₁ = 2 * Δx * ∂ūₗ + uₙ₋₁ for the right hand side of the diffusion equation at i = end 
             rhs[1] += λ * (-4.0 * discretization.Δx * bc.∂ū₀ + 2 * u[2] - 2 * u[1])
             rhs[end] += λ * (4.0 * discretization.Δx * bc.∂ūₗ + 2 * u[end - 1] - 2 * u[end])
         elseif typeof(bc) == Dirichlet
             # Simply input the known boundary condition for Dirichlet
             rhs[1] += λ * (bc.ū₀ - 2 * u[1] + u[2])
             rhs[end] += λ  * (u[end - 1] - 2 * u[end]  + bc.ūₗ)  
         elseif typeof(bc) == Periodic
             # draw a circle to see.
             # u[-1] is actually u[end] by PBCs
             rhs[1] += λ * (u[end] - 2 * u[1] + u[2])
             # u[end+1] is actually end
             rhs[end] += λ * (u[end - 1] - 2 * u[end] + u[1])
         end
      
         # solve for u at next time step (overwrite previous)
         u = A \ rhs
      
         # Store u and t every so often so we can plot. 
         if (k == 1) || (k % sample_freq == 0) 
                u_sample[:, sample_counter] = u
                t_sample[sample_counter] = t 
                sample_counter += 1
         end    

    end
 
#= This chunk of code was the previous way to make rhs and u... can be deleted when we feel appropriate since reformatted above
  
    ## Making right hand side to fill in the rest of u ##
    if typeof(bc) == Dirichlet
        u[1,:] = bc.ū₀
        u[end,:] = bc.ūₗ
        rhs = zeros(Float64, discretization.Nₓ - 2)
        for k = 2:discretization.Nₜ #this is the column of the u array, not the tₖ  
            for i = 2:(discretization.Nₓ - 1) 
                rhs[i-1] = λ * (u[i - 1, k - 1] - 2 * u[i, k - 1] + u[i + 1, k - 1]) + 
                f(x[i], t[k - 1], u[i, k - 1]) * discretization.Δt + u[i, k - 1] 
            end
        u[2:end-1, k] = A \ rhs       #Au = rhs
        end
    end
    
    if typeof(bc) == Periodic 
        rhs = zeros(Float64, discretization.Nₓ - 1)
         for k = 2:discretization.Nₜ #this is the column of the u array, not the tₖ  
            for i = 1:(discretization.Nₓ - 1) 
                if (i == 1)
                    rhs[i] = λ * (u[Nₓ - 1, k - 1] - 2 * u[i, k - 1] + u[i + 1, k - 1]) + f(x[i], t[k - 1], u[i, k - 1]) * discretization.Δt + u[i, k - 1] 
                else
                    rhs[i] = λ * (u[i - 1, k - 1] - 2 * u[i, k - 1] + u[i + 1, k - 1]) + f(x[i], t[k - 1], u[i, k - 1]) * discretization.Δt + u[i, k - 1]
                end
            end
        u[2:end, k] = A \ rhs       #Au = rhs
        end
    end
     
    
    if typeof(bc) == Neumann 
        rhs = zeros(Float64, discretization.Nₓ)
        for k = 2:discretization.Nₜ #this is the column of the u array, not the tₖ  
            for i = 1:(discretization.Nₓ) 
                if (i == 1)
                    rhs[i] = λ * ((-4 * discretization.Δx * bc.∂ū₀) + 2*u[i + 1, k - 1] - 2 * u[i, k - 1]) + f(x[i], t[k - 1], u[i, k - 1]) * discretization.Δt + u[i, k - 1]
                elseif (i == discretization.Nₓ)
                    rhs[i] = λ * ((4 * discretization.Δx * bc.∂ūₗ) + 2*u[i - 1, k - 1] - 2 * u[i, k - 1])  + f(x[i], t[k - 1], u[i, k - 1]) * discretization.Δt + u[i, k - 1]
                else
                    rhs[i] = λ * (u[i - 1] - 2 * u[i] + u[i + 1]) + f(x[i], t, u[i]) * discretization.Δt + u[i, k - 1]
                end
            end
        u[:, k] = A \ rhs       #Au = rhs
        end

    end 
=#
        
    return t_sample, x, u_sample 
    
end 

solve_rxn_diffn_eqn

Enter the following: 

Reaction Term (f(x, t, u))

Initial Condition (u₀)

Boundary Conditions

Diffusion Coefficient (D)

Number of Spacial Steps (Nₓ)

Space Time

Sample Time

In [77]:
#Neumann BC Test
g(x::Float64) = x^3

function exact_u(x::Float64, t::Float64) 
    return e^(-π^2 * t) * cos(π * x) + g(x)
end

f(x::Float64, t::Float64, u::Float64) = -g(x)
u₀(x::Float64) = cos(π * x) + g(x)

bc = Neumann(0.0, 3.0)
D = 1.0
Nₓ = 20
st = SpaceTime(1.0, 1.0)
sample_time = 0.1

t, x, u = solve_rxn_diffn_eqn(f, u₀, bc, D, Nₓ, st, sample_time)

20 points in x-discretization. dx = 0.052632
362 points in t-discretization. dt = 0.002770


([0.00277008, 0.110803, 0.221607, 0.33241, 0.443213, 0.554017, 0.66482, 0.775623, 0.886427, 0.99723], [0.0, 0.0526316, 0.105263, 0.157895, 0.210526, 0.263158, 0.315789, 0.368421, 0.421053, 0.473684, 0.526316, 0.578947, 0.631579, 0.684211, 0.736842, 0.789474, 0.842105, 0.894737, 0.947368, 1.0], [0.973762 0.494381 … 2.22076 2.52526; 0.961017 0.492646 … 2.22457 2.52907; … ; -0.0962921 0.959258 … 3.49209 3.79685; 0.0403178 1.10642 … 3.64479 3.94956])

In [65]:
#Dirichlet BC Test
function exact_u(x::Float64, t::Float64) 
        return 100 * exp(-t) * sin(π * x) #exp(-2 * t) * sin(5 * π * x) #Let L = 1
end

f(x::Float64, t::Float64, u::Float64) = 100 * (D * π^2 - 1.0) * (e ^ (-t)  * sin(π * x))
u₀(x::Float64) = 100 * sin(π * x)

bc = Dirichlet(0.0, 0.0)
D = 1.0
Nₓ = 20
st = SpaceTime(1.0, 1.0)
sample_time = 0.1

t, x, u = solve_rxn_diffn_eqn(f, u₀, bc, D, Nₓ, st, sample_time)

20 points in x-discretization. dx = 0.052632
362 points in t-discretization. dt = 0.002770


([0.00277008, 0.110803, 0.221607, 0.33241, 0.443213, 0.554017, 0.66482, 0.775623, 0.886427, 0.99723], [0.0526316, 0.105263, 0.157895, 0.210526, 0.263158, 0.315789, 0.368421, 0.421053, 0.473684, 0.526316, 0.578947, 0.631579, 0.684211, 0.736842, 0.789474, 0.842105, 0.894737, 0.947368], [16.4144 14.7437 … 6.79117 6.07888; 32.381 29.0852 … 13.3971 11.9919; … ; 32.381 29.0852 … 13.3971 11.9919; 16.4144 14.7437 … 6.79117 6.07888])

In [51]:
u

20×10 Array{Float64,2}:
  0.973762   0.494381  0.529181  …  1.61304  1.91652  2.22076  2.52526
  0.961017   0.492646  0.531129     1.61682  1.92032  2.22457  2.52907
  0.923323   0.487593  0.537022     1.62817  1.93172  2.23598  2.54049
  0.862366   0.479676  0.547015     1.6471   1.95072  2.25501  2.55953
  0.780707   0.469638  0.561363     1.67362  1.97734  2.28167  2.5862 
  0.681598   0.458497  0.580415  …  1.70773  2.01158  2.31596  2.6205 
  0.568875   0.447518  0.604614     1.74948  2.05349  2.35791  2.66247
  0.446869   0.438184  0.634481     1.79891  2.10308  2.40756  2.71214
  0.320307   0.432158  0.670608     1.85608  2.16043  2.46496  2.76956
  0.194208   0.43124   0.713644     1.92105  2.22559  2.53019  2.83481
  0.0737684  0.437322  0.764283  …  1.99393  2.29866  2.60333  2.90797
 -0.0357563  0.452344  0.823248     2.07484  2.37975  2.68448  2.98915
 -0.129169   0.478245  0.891281     2.16391  2.469    2.77379  3.07848
 -0.201452   0.516913  0.969122     2.26131  2.56657 

In [75]:
# Making gif to compare exact equation and rxn_diffn_equ

# Is there a way to use what we already computed in the function? or will we have to return both of these values to do so?
Δx = x[2] - x[1]
Δt = Δx ^ 2

Nₜ = ceil(Int, st.tf / sample_time)

@gif for i = 1:Nₜ
    scatter(x,u[:,i], ylim=(0,100), label="numerical", xlabel="x", ylabel="u", title=@sprintf("t = %.2f", t[i]))
    plot!(x, exact_u.(x,t[i]), label="exact")
end 

LoadError: [91mcould not spawn `ffmpeg -v 0 -i 'C:\Users\Rachel\AppData\Local\Temp\jl_41D3.tmp/%06d.png' -vf palettegen=stats_mode=diff -y 'C:\Users\Rachel\AppData\Local\Temp\jl_41D3.tmp/palette.bmp'`: no such file or directory (ENOENT)[39m

In [76]:
# Use this to make heat maps
import PyPlot; const plt = PyPlot

PyPlot

In [46]:
"""
Creates heat map based on x, t, u
"""
function draw_heat_map(x, t, u)
    Nₜ = length(t)
    Nₓ = length(x)
    
    X = similar(transpose(u))
    T = similar(transpose(u))

    for i = 1:Nₜ
        X[i,:] = x
    end

    for i = 1:Nₓ
        T[:,i] = t
    end

    plt.figure()
    plt.pcolormesh(X,T, transpose(u))
    plt.xlabel("x")
    plt.ylabel("t")
    plt.colorbar(label="Concentration of A")
    plt.savefig("heatmap.png")
end 

draw_heat_map (generic function with 1 method)

In [68]:
# Making heat map based off of calulated u
draw_heat_map(x, t, u)

In [66]:
# Making heat map based off of exact u to compare against calculated u heat map
Nₜ = length(t)
Nₓ = length(x)

u_e = zeros(Float64, Nₓ, Nₜ)
for i = 1:Nₓ 
    for j = 1:Nₜ
        u_e[i,j] = exact_u(x[i], t[j])
    end
end

draw_heat_map(x, t, u_e)